In [19]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, BatchNormalization, LocallyConnected2D, SeparableConv2D
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras.constraints import max_norm
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import activations

In [20]:
from create_data import create_all_x_y

In [21]:
X, y = create_all_x_y()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

8 parties rejetées par le goban, reste 2294 parties
(14681, 9, 9, 49)
(14681, 82)


In [28]:
model = Sequential()
model.add(Conv2D(32, kernel_size=5, activation='relu', input_shape=X[0].shape, padding ="same"))
model.add(Conv2D(256, kernel_size=5, activation='relu', padding ="same"))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(82, activation='relu')) 
opt= optimizers.Adam()
model.compile(loss='mean_absolute_error',optimizer = opt)
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 9, 9, 32)          39232     
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 9, 9, 256)         205056    
_________________________________________________________________
flatten_6 (Flatten)          (None, 20736)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               5308672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 82)                21074     
Total params: 5,574,034
Trainable params: 5,574,034
Non-trainable params: 0
____________________________________________

In [29]:
# hyperparamètres
epochs = 50
batch_size = 64

# training
print("Training ...")
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Training ...
Epoch 1/50
230/230 [==============================] - 23s 99ms/step - loss: 0.2615 - val_loss: 0.2047
Epoch 2/50
230/230 [==============================] - 22s 98ms/step - loss: 0.2141 - val_loss: 0.2033
Epoch 3/50
230/230 [==============================] - 22s 98ms/step - loss: 0.2084 - val_loss: 0.2040
Epoch 4/50
230/230 [==============================] - 24s 103ms/step - loss: 0.2065 - val_loss: 0.2028
Epoch 5/50
230/230 [==============================] - 24s 104ms/step - loss: 0.2049 - val_loss: 0.2040
Epoch 6/50
230/230 [==============================] - 22s 97ms/step - loss: 0.2034 - val_loss: 0.2029
Epoch 7/50
230/230 [==============================] - 19s 82ms/step - loss: 0.2022 - val_loss: 0.2035
Epoch 8/50
230/230 [==============================] - 19s 82ms/step - loss: 0.2013 - val_loss: 0.2048
Epoch 9/50
230/230 [==============================] - 19s 83ms/step - loss: 0.2006 - val_loss: 0.2031
Epoch 10/50
230/230 [==============================] - 19s 81ms/ste

In [30]:
proba = model.predict(X_train)

In [31]:
print(proba[5])

[0.7282461  0.71484053 0.6941715  0.7409907  0.70451    0.7350548
 0.72047114 0.70879865 0.7215136  0.7278557  0.70899785 0.74537796
 0.7204546  0.6973832  0.70884585 0.71870995 0.7253165  0.7125609
 0.74284387 0.68233067 0.6780529  0.699715   0.71877724 0.7211054
 0.7056892  0.67990583 0.6987027  0.73014843 0.70430374 0.6972923
 0.719298   0.701437   0.70477253 0.6777748  0.7021182  0.745227
 0.71740425 0.6827823  0.7357174  0.69521403 0.7539677  0.6891929
 0.6933629  0.706938   0.7094521  0.7605224  0.7168398  0.7025902
 0.6965906  0.6961489  0.6973293  0.70584756 0.721261   0.72199166
 0.73237276 0.7278678  0.72395086 0.6544146  0.7215734  0.68234724
 0.6939244  0.6989778  0.7084781  0.70656604 0.6999239  0.71808267
 0.7141762  0.7497333  0.7283159  0.75045264 0.7187079  0.74589354
 0.6984762  0.74149054 0.74054986 0.76183695 0.7380772  0.7440549
 0.7395382  0.7329122  0.7190939  0.70207286]
